# Análise Estatística


## Testes Paramétricos

Nesta etapa realizamos os testes de normalidade de Shapiro-Wilk.

In [24]:
import pandas as pd
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Carregar arquivo Excel
excel_file_path = 'metrics_results.xlsx'
excel_data = pd.ExcelFile(excel_file_path)

# Dicionário para armazenar os valores extraídos
extracted_values = {}

# Ler dados de cada aba e extrair as 10 primeiras linhas de cada classificador
for sheet_name in excel_data.sheet_names:
    sheet_df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
    classifiers = ['KNN', 'Decision Tree', 'Random Forest', 'XGBoost', 'LightGBM']
    for classifier in classifiers:
        extracted_values[f"{sheet_name}_{classifier}"] = sheet_df[classifier].iloc[:10].tolist()

# Realizar teste de Shapiro-Wilk para cada métrica e classificador
shapiro_results = {}

for metric in ['Accuracy', 'F1 Score', 'Recall', 'ACSA']:
    shapiro_results[metric] = {}
    for classifier in classifiers:
        # Obter os dados
        data = extracted_values[f"{metric}_{classifier}"]
        
        # Realizar o teste de Shapiro-Wilk
        statistic, p_value = stats.shapiro(data)
        
        # Armazenar resultados
        shapiro_results[metric][classifier] = {
            'Statistic': statistic,
            'P-value': p_value,
            'Is Normal': p_value > 0.05  # Considerando alfa = 0.05
        }

# Criar DataFrame com os resultados
results_data = []
for metric in shapiro_results:
    for classifier in shapiro_results[metric]:
        results_data.append({
            'Metric': metric,
            'Classifier': classifier,
            'Statistic': shapiro_results[metric][classifier]['Statistic'],
            'P-value': shapiro_results[metric][classifier]['P-value'],
            'Is Normal': shapiro_results[metric][classifier]['Is Normal']
        })

results_df = pd.DataFrame(results_data)

# Formatar o DataFrame para melhor visualização
results_df['Statistic'] = results_df['Statistic'].round(4)
results_df['P-value'] = results_df['P-value'].round(4)

# Salvar resultados em CSV
results_df.to_csv('shapiro_wilk_results.csv', index=False)
print("Resultados do teste de Shapiro-Wilk:")
print(results_df)

# Criar um resumo por métrica
print("\nResumo da normalidade por métrica:")
for metric in ['Accuracy', 'F1 Score', 'Recall', 'ACSA']:
    normal_count = len(results_df[(results_df['Metric'] == metric) & (results_df['Is Normal'])])
    total_count = len(results_df[results_df['Metric'] == metric])
    print(f"\n{metric}:")
    print(f"- Distribuições normais: {normal_count} de {total_count} classificadores")
    print("- Classificadores com distribuição normal:")
    normal_classifiers = results_df[(results_df['Metric'] == metric) & (results_df['Is Normal'])]['Classifier'].tolist()
    if normal_classifiers:
        for clf in normal_classifiers:
            print(f"  * {clf}")
    else:
        print("  * Nenhum classificador apresentou distribuição normal")


Resultados do teste de Shapiro-Wilk:
      Metric     Classifier  Statistic  P-value  Is Normal
0   Accuracy            KNN     0.9674   0.8660       True
1   Accuracy  Decision Tree     0.9226   0.3789       True
2   Accuracy  Random Forest     0.9513   0.6835       True
3   Accuracy        XGBoost     0.9200   0.3566       True
4   Accuracy       LightGBM     0.9017   0.2289       True
5   F1 Score            KNN     0.9664   0.8559       True
6   F1 Score  Decision Tree     0.9171   0.3331       True
7   F1 Score  Random Forest     0.9553   0.7316       True
8   F1 Score        XGBoost     0.9203   0.3591       True
9   F1 Score       LightGBM     0.9005   0.2220       True
10    Recall            KNN     0.9542   0.7187       True
11    Recall  Decision Tree     0.8725   0.1069       True
12    Recall  Random Forest     0.9484   0.6498       True
13    Recall        XGBoost     0.9752   0.9348       True
14    Recall       LightGBM     0.8781   0.1241       True
15      ACSA       

## Testes Não Paramétricos

Nesta etapa realizamos os teste de Friedman, post hoc Niemenyi e mapas de calor.

In [22]:
import scikit_posthocs as sp
import numpy as np
import pandas as pd
from scipy.stats import friedmanchisquare
import seaborn as sns
import matplotlib.pyplot as plt

# Carregar arquivo Excel
excel_file_path = 'metrics_results.xlsx'
excel_data = pd.ExcelFile(excel_file_path)

# Dicionário para armazenar os valores extraídos
extracted_values = {}

# Ler dados de cada aba e extrair as 10 primeiras linhas de cada classificador
for sheet_name in excel_data.sheet_names:
    sheet_df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
    classifiers = ['KNN', 'Decision Tree', 'Random Forest', 'XGBoost', 'LightGBM']
    for classifier in classifiers:
        extracted_values[f"{sheet_name}_{classifier}"] = sheet_df[classifier].iloc[:10].tolist()

# Dicionário com as métricas e seus respectivos valores
metrics = {
    "Accuracy": [extracted_values['Accuracy_KNN'], 
                 extracted_values['Accuracy_Decision Tree'],
                 extracted_values['Accuracy_Random Forest'],
                 extracted_values['Accuracy_XGBoost'],
                 extracted_values['Accuracy_LightGBM']],
    
    "F1 Score": [extracted_values['F1 Score_KNN'],
                 extracted_values['F1 Score_Decision Tree'],
                 extracted_values['F1 Score_Random Forest'],
                 extracted_values['F1 Score_XGBoost'],
                 extracted_values['F1 Score_LightGBM']],
    
    "Recall": [extracted_values['Recall_KNN'],
               extracted_values['Recall_Decision Tree'],
               extracted_values['Recall_Random Forest'],
               extracted_values['Recall_XGBoost'],
               extracted_values['Recall_LightGBM']],
    
    "ACSA": [extracted_values['ACSA_KNN'],
             extracted_values['ACSA_Decision Tree'],
             extracted_values['ACSA_Random Forest'],
             extracted_values['ACSA_XGBoost'],
             extracted_values['ACSA_LightGBM']]
}

# Nomes dos classificadores
classifiers = ['KNN', 'Decision Tree', 'Random Forest', 'XGBoost', 'LightGBM']

# Primeiro executar o teste de Friedman para cada métrica
friedman_results = {}
for metric, data in metrics.items():
    stat, p_value = friedmanchisquare(*data)
    friedman_results[metric] = {"Statistic": stat, "P-value": p_value}

# Executar o teste pós-hoc de Nemenyi para métricas significativas
posthoc_results = {}
for metric, data in metrics.items():
    # Verificar se o teste de Friedman indicou diferença significativa
    if friedman_results[metric]["P-value"] < 0.05:
        data_transposed = np.array(data).T  # Transpor os dados
        nemenyi = sp.posthoc_nemenyi_friedman(data_transposed)
        nemenyi.index = classifiers
        nemenyi.columns = classifiers
        posthoc_results[metric] = nemenyi

        # Exportar resultados do teste de Nemenyi para CSV
        nemenyi.to_csv(f'nemenyi_results_{metric}.csv')
        print(f"Resultados do teste de Nemenyi para {metric} exportados para 'nemenyi_results_{metric}.csv'.")

        # Criar e salvar o heatmap
        plt.figure(figsize=(10, 8))
        sns.heatmap(
            nemenyi,
            annot=True,
            fmt=".3f",
            cmap="coolwarm",
            cbar_kws={'label': 'Valor-p'},
            xticklabels=nemenyi.columns,
            yticklabels=nemenyi.index
        )
        plt.title(f'Mapa de Calor - Teste de Nemenyi ({metric})', fontsize=16)
        plt.xlabel('Classificadores', fontsize=12)
        plt.ylabel('Classificadores', fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        # Salvar mapa de calor como imagem
        heatmap_path = f'nemenyi_heatmap_{metric}.png'
        plt.savefig(heatmap_path)
        print(f"Mapa de calor para {metric} salvo como '{heatmap_path}'.")
        plt.close()  # Fechar a figura para liberar memória

# Mostrar resultados do teste de Friedman
print("\nResultados do teste de Friedman:")
friedman_df = pd.DataFrame.from_dict(friedman_results, orient='index')
print(friedman_df)


Resultados do teste de Nemenyi para Accuracy exportados para 'nemenyi_results_Accuracy.csv'.
Mapa de calor para Accuracy salvo como 'nemenyi_heatmap_Accuracy.png'.
Resultados do teste de Nemenyi para F1 Score exportados para 'nemenyi_results_F1 Score.csv'.
Mapa de calor para F1 Score salvo como 'nemenyi_heatmap_F1 Score.png'.
Resultados do teste de Nemenyi para Recall exportados para 'nemenyi_results_Recall.csv'.
Mapa de calor para Recall salvo como 'nemenyi_heatmap_Recall.png'.
Resultados do teste de Nemenyi para ACSA exportados para 'nemenyi_results_ACSA.csv'.
Mapa de calor para ACSA salvo como 'nemenyi_heatmap_ACSA.png'.

Resultados do teste de Friedman:
          Statistic       P-value
Accuracy      38.08  1.078775e-07
F1 Score      38.08  1.078775e-07
Recall        38.56  8.587571e-08
ACSA          38.00  1.120559e-07


## Média e Desvio Padrão

In [25]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Carregar arquivo Excel
excel_file_path = 'metrics_results.xlsx'
excel_data = pd.ExcelFile(excel_file_path)

# Dicionário para armazenar os valores extraídos
extracted_values = {}

# Ler dados de cada aba e extrair as 10 primeiras linhas de cada classificador
for sheet_name in excel_data.sheet_names:
    sheet_df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
    classifiers = ['KNN', 'Decision Tree', 'Random Forest', 'XGBoost', 'LightGBM']
    for classifier in classifiers:
        extracted_values[f"{sheet_name}_{classifier}"] = sheet_df[classifier].iloc[:10].tolist()

# Calcular estatísticas para cada métrica e classificador
stats_results = {
    'Accuracy_F1': [],
    'Recall_ACSA': []
}

# Lista de classificadores
classifiers = ['KNN', 'Decision Tree', 'Random Forest', 'XGBoost', 'LightGBM']

# Calcular estatísticas para Accuracy e F1 Score
for classifier in classifiers:
    accuracy_data = extracted_values[f"Accuracy_{classifier}"]
    f1_data = extracted_values[f"F1 Score_{classifier}"]
    
    stats_results['Accuracy_F1'].append({
        'Classificador': classifier,
        'Média Accuracy': np.mean(accuracy_data),
        'Desvio Padrão Accuracy': np.std(accuracy_data),
        'Média F1 Score': np.mean(f1_data),
        'Desvio Padrão F1 Score': np.std(f1_data)
    })

# Calcular estatísticas para Recall e ACSA
for classifier in classifiers:
    recall_data = extracted_values[f"Recall_{classifier}"]
    acsa_data = extracted_values[f"ACSA_{classifier}"]
    
    stats_results['Recall_ACSA'].append({
        'Classificador': classifier,
        'Média Recall': np.mean(recall_data),
        'Desvio Padrão Recall': np.std(recall_data),
        'Média ACSA': np.mean(acsa_data),
        'Desvio Padrão ACSA': np.std(acsa_data)
    })

# Criar DataFrames
df_accuracy_f1 = pd.DataFrame(stats_results['Accuracy_F1'])
df_recall_acsa = pd.DataFrame(stats_results['Recall_ACSA'])

# Arredondar valores para 4 casas decimais
columns_to_round = [col for col in df_accuracy_f1.columns if col != 'Classificador']
df_accuracy_f1[columns_to_round] = df_accuracy_f1[columns_to_round].round(4)

columns_to_round = [col for col in df_recall_acsa.columns if col != 'Classificador']
df_recall_acsa[columns_to_round] = df_recall_acsa[columns_to_round].round(4)

# Criar arquivo Excel com as duas abas
with pd.ExcelWriter('metricas_estatisticas.xlsx') as writer:
    df_accuracy_f1.to_excel(writer, sheet_name='Accuracy e F1 Score', index=False)
    df_recall_acsa.to_excel(writer, sheet_name='Recall e ACSA', index=False)

print("Arquivo 'metricas_estatisticas.xlsx' foi criado com sucesso!")
print("\nAba 1 - Accuracy e F1 Score:")
print(df_accuracy_f1)
print("\nAba 2 - Recall e ACSA:")
print(df_recall_acsa)


Arquivo 'metricas_estatisticas.xlsx' foi criado com sucesso!

Aba 1 - Accuracy e F1 Score:
   Classificador  Média Accuracy  Desvio Padrão Accuracy  Média F1 Score  \
0            KNN          0.9766                  0.0003          0.9763   
1  Decision Tree          0.9737                  0.0009          0.9735   
2  Random Forest          0.9581                  0.0011          0.9568   
3        XGBoost          0.9924                  0.0004          0.9925   
4       LightGBM          0.9926                  0.0005          0.9927   

   Desvio Padrão F1 Score  
0                  0.0003  
1                  0.0009  
2                  0.0012  
3                  0.0004  
4                  0.0005  

Aba 2 - Recall e ACSA:
   Classificador  Média Recall  Desvio Padrão Recall  Média ACSA  \
0            KNN        0.9545                0.0005      0.9979   
1  Decision Tree        0.9574                0.0017      0.9819   
2  Random Forest        0.9221                0.0024    